In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import os
import pandas as pd
from glob import glob
import shutil
from tqdm import tqdm
import cv2
from natsort import natsorted

from keras_self_attention import SeqSelfAttention
from keras.models import load_model
from keras.utils import np_utils
np.seterr(divide='ignore', invalid='ignore')
from utils import noise_feature

In [14]:
model = load_model("./model/trained_model.h5", custom_objects={'SeqSelfAttention': SeqSelfAttention})
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 10, 128)           2048      
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 128)           512       
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 10, 128)           0         
_________________________________________________________________
seq_self_attention_1 (SeqSel (None, 10, 128)           8257      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [15]:
inputFolder = ['awgn', 'defocus_blur', 'motion_blur', 'smoke', 'uneven_illum', 'defocus_uneven', 'noise_smoke',               
                'noise_smoke_uneven', 'noise_uneven', 'smoke_uneven']
labelDict = dict()
for count, label in enumerate(inputFolder):
    labelDict[label] = count

In [64]:
dataset_location = './demo data'
videos = natsorted(os.listdir(dataset_location))

extracted_location = './Extracted_demo_data'
if not os.path.exists(extracted_location):
    os.mkdir(extracted_location)

demo_result = './demo result'
if not os.path.exists(demo_result):
    os.mkdir(demo_result)
    
sequence_length = 10
test_labelname = []
y_pred = []


sca = np.load('./scale_data.npy',allow_pickle='TRUE')


for video in videos:
    videoDir = os.path.join(dataset_location, video)
    outDir = os.path.join(extracted_location, os.path.splitext(video)[0])
    if not os.path.exists(outDir):
        os.mkdir(outDir)
    os.system('ffmpeg -i "{}" -vf "select=not(mod(n\,20))" -vsync vfr {}/%01d.png'.format(videoDir, outDir))

    
    images = natsorted(glob(os.path.join(outDir, '*.png')))
    xListSingleFrame = []
    count = 0
    for j in range(len(images)):
        d = cv2.imread(images[j])
        lab = cv2.cvtColor(d, cv2.COLOR_BGR2LAB)
        lab_planes = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=5.0)
        lab_planes[0] = clahe.apply(lab_planes[0])
        lab = cv2.merge(lab_planes)
        d = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        d = cv2.resize(d, (128,128))
        
        xListSingleFrame.append(noise_feature(d))

        del(d)
        if (count+1) == sequence_length:
            x_test= np.asarray(xListSingleFrame)[None]
            for k in range(x_test.shape[-1]):
                minV, maxV = sca[k]
                xx = x_test[:,:,k]
                xx[xx>maxV] = maxV
                xx[xx<minV] = minV
                x_test[:,:,k] = xx
                x_test[:,:,k] = (x_test[:,:,k] - minV)/(maxV-minV)
            text = list(labelDict.keys())[model.predict(x_test).argmax(axis = 1)[0]]
            demo_video_name = os.path.join(demo_result,'demo_result_of_' + video)
            if os.path.exists(demo_video_name):
                os.remove(demo_video_name)
            os.system('ffmpeg -i "{}" -filter_complex drawtext=text="{}":fontcolor=white:fontsize=30:x=(w-text_w)/2:box=1:boxcolor=black -codec:a copy "{}"'.format(videoDir, text, demo_video_name))
            print('Detected distortion of', video,'is: ',text)
            y_pred.append(list(labelDict.keys())[model.predict(x_test).argmax(axis = 1)[0]])
            test_labelname.append(video)
            break
        count += 1
shutil.rmtree(extracted_location)

print("""

The output video of this demo code is stored in 'demo result' directory

""")

Detected distortion of video1_1.avi is:  awgn
Detected distortion of video2_1.avi is:  defocus_blur
Detected distortion of video3_1.avi is:  defocus_blur
Detected distortion of video4_1.avi is:  motion_blur
Detected distortion of video5_1.avi is:  noise_smoke
Detected distortion of video6_1.avi is:  noise_smoke_uneven
Detected distortion of video7_1.avi is:  noise_uneven
Detected distortion of video8_1.avi is:  smoke
Detected distortion of video9_1.avi is:  smoke
Detected distortion of video10_1.avi is:  uneven_illum


The output video of this demo code is stored in 'demo result' directory


